In [8]:
from IPython.display import display
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.csg import *

In [9]:
# 1. Define the Geometry
geo = CSGeometry()
box = OrthoBrick(Pnt(-1, -1, -1), Pnt(1, 1, 1))
geo.Add(box)
mesh = Mesh(geo.GenerateMesh(maxh=0.2))

In [10]:
# 2. Define the Finite Element Spaces
# Separate HCurl and HDiv spaces
V_k = HCurl(mesh, order=1)   
V_km1 = HDiv(mesh, order=1)  

productSpace = V_k * V_km1

# Trial and test functions for product space (HCurl and HDiv components)
sigma_h, omega_h = productSpace.TrialFunction()
tau_h, eta_h = productSpace.TestFunction()

# # 1. Compute the minimum mesh size (hmin)
# 1. Compute the minimum mesh size (hmin)
h_min = min(Integrate(1, mesh, BND, element_wise=True))
print(h_min)

0.011374548767673562


In [11]:
# 3. Bilinear forms for HCurl and HDiv spaces

# HCurl part (1-forms)
a_hcurl = BilinearForm(productSpace)
a_hcurl += SymbolicBFI(InnerProduct(sigma_h, tau_h))

# HDiv part (2-forms)
a_hdiv = BilinearForm(productSpace)
a_hdiv += SymbolicBFI(InnerProduct(curl(sigma_h), eta_h))  # Correct use of curl from HCurl to HDiv
a_hdiv += SymbolicBFI(InnerProduct(div(omega_h), div(eta_h)))

# 4. Boundary integrals
# Boundary Integral 1: 
boundary_integral_1 = BilinearForm(productSpace)
boundary_integral_1 += InnerProduct(div(omega_h), eta_h.Trace()) * ds

# Boundary Integral 2: 
boundary_integral_2 = BilinearForm(productSpace)
boundary_integral_2 += omega_h * div(eta_h) * ds


# Penalty term: 
C_w = 1.0
penalty_term = BilinearForm(productSpace)
penalty_term += (C_w / h_min) * omega_h.Trace() * eta_h.Trace() * dx


NgException: T_MultVecVec : dimensions don't fit

In [ ]:
# Assemble all bilinear forms (HCurl, HDiv, boundary terms, penalty term)
a_hcurl.Assemble()
a_hdiv.Assemble()
boundary_integral_1.Assemble()
boundary_integral_2.Assemble()
penalty_term.Assemble()

# Sum the matrices from all the bilinear forms to create a global matrix
total_bilinear_form_matrix = a_hcurl.mat + a_hdiv.mat + boundary_integral_1.mat + boundary_integral_2.mat + penalty_term.mat
#total_bilinear_form_matrix = a_hcurl.mat + a_hdiv.mat
